In [1]:
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, Dense
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import pandas as pd
import re
import keras
import numpy as np
import scipy
import copy

Using TensorFlow backend.


In [2]:
#master_data = pd.read_csv("multilabels.csv")
#master_data = pd.read_csv("train.csv")
master_data = pd.read_csv("multiclass.csv")

In [3]:
features = np.array(master_data.iloc[:, 1])

In [4]:
labels = np.array(master_data.iloc[:, 0])

In [5]:
test = features
p_data = []

for eachPoint in test:
    p_data.append([re.sub(r"(http|ftp|https)\S+", "", str(eachPoint))])

In [6]:
test = p_data

In [7]:
def lowercase(arr):
    return np.char.lower(arr)

In [8]:
def removeStopWords(arr):
    stopWords = set(stopwords.words('english'))
    
    filteredWords = []
    for a in arr:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            if w not in stopWords:
                wordsFiltered = wordsFiltered + " " + w
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [9]:
def removeSymbols(arr):
    symbols = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    for i in range(len(symbols)):
        arr = np.char.replace(arr, symbols[i], '')   
    return np.char.replace(arr, "  ", " ")

In [10]:
def stemming(word):
    porter_stemmer=PorterStemmer()
    filteredWords = []
    for a in word:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            wordsFiltered = wordsFiltered + " " + porter_stemmer.stem(w)
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [11]:
def removeImplicitNoise(arr):
    return np.char.replace (arr, 'NEWLINE_TOKEN', '')

In [12]:
def processClean(data):
    a = removeImplicitNoise(data)
    a = lowercase(a)
    a = removeStopWords(a)
    a = removeSymbols(a)
    a = stemming(a)
    return a

a = processClean(test)

In [13]:
list_l = []

for i in a:
    list_l.append(str(i[0]))

In [14]:
def make_grams(data, n = 2):
    
    grammed_data = []
    
    for i in data:

        k = copy.deepcopy(i)

        for r in range(2,n+1):
            sixgrams = ngrams(i.split(), r)

            for grams in sixgrams:
                g = ""
                for p in grams:
                    g = g+p

                k = k+" "+g
        grammed_data.append(k)
    
    return grammed_data

grammed_data = make_grams(list_l)

In [15]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(feature_vector.shape)

(4500, 81065)


In [16]:
# TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(vector.shape)

(4500, 81065)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.10, random_state=42)

In [18]:
from keras.utils import to_categorical

data = y_train
print(data)

def encode(data):
    encoded = to_categorical(data,dtype='int')
    return encoded

encoded_data = encode(data)
print(encoded_data)

[0 2 0 ... 2 2 0]
[[1 0 0]
 [0 0 1]
 [1 0 0]
 ...
 [0 0 1]
 [0 0 1]
 [1 0 0]]


In [19]:
data = y_test

def encode(data):
    encoded = to_categorical(data,dtype='int')
    return encoded

encoded_data_test = encode(data)

In [20]:
top_words = 10000

max_review_length = 2000

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 50

model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='exponential'))
model.add(MaxPooling1D(pool_size=2))
model.add(CuDNNLSTM(18))
model.add(Dense(3, activation='softmax'))

opt = keras.optimizers.SGD(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

model.fit(X_train[0:10000], encoded_data[0:10000], epochs=10, batch_size=64)

scores = model.evaluate(X_test[0:10000], encoded_data_test[0:10000], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          500000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2000, 32)          4832      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1000, 32)          0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 18)                3744      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 57        
Total params: 508,633
Trainable params: 508,633
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
4050/4050 [==============================] - 5s 1ms/step - loss: 0.6386 - acc: 0.6667
Epoch 2/10
4050/4050 [============

In [25]:
text_labels=encode(y_test)
y_predicted=[]
for i in range(len(X_test)):
    prediction = model.predict(np.array([X_test[i]]))
    predicted_label = np.argmax(prediction[0])
    y_predicted.append(predicted_label)

print(f1_score(y_test,y_predicted,average=None))
print(classification_report(y_test,y_predicted))

[0.         0.         0.52459016]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       152
          1       0.00      0.00      0.00       138
          2       0.36      1.00      0.52       160

avg / total       0.13      0.36      0.19       450



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
from sklearn.metrics import f1_score, classification_report